In [47]:
import pandas as pd
import os
from sklearn import preprocessing
from collections import deque
import random
import numpy as np
import time

# Longitud de la secuencia precedente para recolectar para la RNN
SEQ_LEN = 60

# Cuánto tiempo en el futuro estamos tratando de predecir
FUTURE_PERIOD_PREDICT = 8

# El ratio que queremos predecir
RATIO_TO_PREDICT = "BCH-USD"

EPOCHS = 10  # cuántas pasadas a través de nuestros datos
BATCH_SIZE = 64  # ¿cuántos lotes? Intenta con un tamaño de lote más pequeño si estás recibiendo errores OOM (fuera de memoria).
NAME = f"{RATIO_TO_PREDICT}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"  # un nombre único para el modelo

# Función para clasificar si el valor futuro es mayor que el valor actual
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

def preprocess_df(df):
    df = df.drop(["future"], axis=1)  # Elimina la columna "future" que ya no necesitamos.

    for col in df.columns:  # Recorre todas las columnas
        if col != "target":  # Normaliza todas las columnas excepto el objetivo mismo
            df[col] = df[col].pct_change()  # Calcula el cambio porcentual (pct_change) de cada columna
            df.dropna(inplace=True)  # Elimina las filas con valores nulos generados por pct_change
            df[col] = preprocessing.scale(df[col].values)  # Escala los valores entre 0 y 1.

    df.dropna(inplace=True)  # Limpieza adicional para asegurarse de no tener valores nulos
    sequential_data = []  # Lista que contendrá las secuencias
    prev_days = deque(maxlen=SEQ_LEN)  # Estas serán nuestras secuencias reales. Se crean con deque, que mantiene la longitud máxima.

    for i in df.values:  # Itera sobre los valores
        prev_days.append([n for n in i[:-1]])  # Almacena todo menos el objetivo
        if len(prev_days) == SEQ_LEN:  # Asegura que tengamos 60 secuencias
            sequential_data.append([np.array(prev_days), i[-1]])  # ¡Añade esas secuencias!

    random.shuffle(sequential_data)  # Baraja las secuencias para mayor variedad.

    buys = []  # Lista que almacenará nuestras secuencias de compra y objetivos
    sells = []  # Lista que almacenará nuestras secuencias de venta y objetivos

    for seq, target in sequential_data:  # Itera sobre los datos secuenciales
        if target == 0:  # Si es "no comprar"
            sells.append([seq, target])  # Agrega a la lista de ventas
        elif target == 1:  # Si el objetivo es 1...
            buys.append([seq, target])  # ¡Es una compra!

    random.shuffle(buys)  # Baraja las compras
    random.shuffle(sells)  # Baraja las ventas

    lower = min(len(buys), len(sells))  # ¿Cuál es la longitud más corta?

    buys = buys[:lower]  # Asegura que ambas listas tengan la misma longitud.
    sells = sells[:lower]  # Asegura que ambas listas tengan la misma longitud.

    sequential_data = buys + sells  # Combínalas
    random.shuffle(sequential_data)  # Otra mezcla, para que el modelo no se confunda con una clase seguida de la otra.

    X = []
    y = []

    for seq, target in sequential_data:  # Recorre los datos secuenciales
        X.append(seq)  # X son las secuencias
        y.append(target)  # y son los objetivos/etiquetas (compras vs no compra)

    return np.array(X), y  # Devuelve X e y, y convierte X en un array de numpy

# Ruta del archivo de datos
file_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\crypto_data\BTC-USD.csv"

# Leer el archivo inicial
df = pd.read_csv(file_path, names=['time', 'low', 'high', 'open', 'close', 'volume'])

# DataFrame principal para almacenar todos los datos
main_df = pd.DataFrame()

# Las 4 criptomonedas que queremos considerar
ratios = ["BCH-USD", "BTC-USD", "ETH-USD", "LTC-USD"]

# Ruta base donde se encuentran los archivos de datos
base_path = r"C:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\Modelo Para Predecir Criptomonedas\crypto_data"

# Iterar sobre cada ratio
for ratio in ratios:
    print(ratio)
    
    # Obtener la ruta completa al archivo
    dataset = os.path.join(base_path, f"{ratio}.csv")
    
    # Leer el archivo específico
    df = pd.read_csv(dataset, names=['time', 'low', 'high', 'open', 'close', 'volume'])
    
    # Renombrar columnas de 'close' y 'volume' para incluir el ticker
    df.rename(columns={"close": f"{ratio}_close", "volume": f"{ratio}_volume"}, inplace=True)
    
    # Establecer 'time' como índice para poder unirlos por este tiempo compartido
    df.set_index("time", inplace=True)
    
    # Ignorar otras columnas además de 'close' y 'volume'
    df = df[[f"{ratio}_close", f"{ratio}_volume"]]
    
    # Si el DataFrame principal está vacío, simplemente asignar el DataFrame actual
    if len(main_df) == 0:
        main_df = df
    else:
        # De lo contrario, unir estos datos con el DataFrame principal
        main_df = main_df.join(df)

main_df.fillna(method="ffill", inplace=True)  # Rellena los huecos de los datos usando los valores conocidos previos
main_df.dropna(inplace=True)  # Elimina cualquier fila que aún tenga valores nulos

# Crear una nueva columna 'future' que es el valor de cierre futuro desplazado por 'FUTURE_PERIOD_PREDICT'
main_df['future'] = main_df[f'{RATIO_TO_PREDICT}_close'].shift(-FUTURE_PERIOD_PREDICT)

# Crear una columna 'target' que clasifica si el valor futuro es mayor que el valor actual
main_df['target'] = list(map(classify, main_df[f'{RATIO_TO_PREDICT}_close'], main_df['future']))

main_df.dropna(inplace=True)  # Elimina cualquier fila que aún tenga valores nulos

# Imprimir las primeras filas del DataFrame resultante
#print(main_df.head())


BCH-USD
BTC-USD
ETH-USD
LTC-USD


C:\Users\CRISTIAN CHAVEZ\AppData\Local\Temp\ipykernel_27544\441976918.py:119: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  main_df.fillna(method="ffill", inplace=True)  # Rellena los huecos de los datos usando los valores conocidos previos


In [48]:
# Obtener una lista ordenada de los valores del índice (los tiempos)
times = sorted(main_df.index.values)

# Calcular el punto de corte para el último 5% de los tiempos
last_5pct = sorted(main_df.index.values)[-int(0.05 * len(times))]

# Imprimir el valor del tiempo que representa el inicio del último 5% del conjunto de datos
print(last_5pct)


1534879440


In [49]:
# Crear el DataFrame de validación donde el índice es mayor o igual al punto de corte del último 5%
validation_main_df = main_df[(main_df.index >= last_5pct)]

# Actualizar main_df para que contenga solo los datos hasta antes del último 5%
main_df = main_df[(main_df.index < last_5pct)]

# Preprocesar el DataFrame principal para obtener los datos de entrenamiento
train_x, train_y = preprocess_df(main_df)

# Preprocesar el DataFrame de validación para obtener los datos de validación
validation_x, validation_y = preprocess_df(validation_main_df)

# Imprimir la cantidad de datos de entrenamiento y validación
print(f"train data: {len(train_x)} validation: {len(validation_x)}")

# Imprimir la cantidad de etiquetas 'no compra' (0) y 'compra' (1) en los datos de entrenamiento
print(f"Dont buys: {train_y.count(0)}, buys: {train_y.count(1)}")

# Imprimir la cantidad de etiquetas 'no compra' (0) y 'compra' (1) en los datos de validación
print(f"VALIDATION Dont buys: {validation_y.count(0)}, buys: {validation_y.count(1)}")


train data: 82408 validation: 4134
Dont buys: 41204, buys: 41204
VALIDATION Dont buys: 2067, buys: 2067


In [50]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, BatchNormalization, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# Assuming train_x, train_y, validation_x, validation_y are lists
train_x = np.array(train_x)
train_y = np.array(train_y)
validation_x = np.array(validation_x)
validation_y = np.array(validation_y)

# Check the shapes of your data
print(f'train_x shape: {train_x.shape}')
print(f'train_y shape: {train_y.shape}')
print(f'validation_x shape: {validation_x.shape}')
print(f'validation_y shape: {validation_y.shape}')

# Model definition
model = Sequential()
model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.1))
model.add(BatchNormalization())

model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))

# Optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# Callbacks
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
filepath = "models/RNN_Final-{epoch:02d}-{val_accuracy:.3f}.keras"  # Use '.keras' extension
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train model
history = model.fit(
    train_x, train_y,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x, validation_y),
    callbacks=[tensorboard, checkpoint],
)

#Ejecutamos el siguiente comando tensorboard --logdir=logs/
#en la ruta donde se tiene guardado el codigo
#Despues de abrir nuestro localhost, se va a scalars y escribimos \w

train_x shape: (82408, 60, 8)
train_y shape: (82408,)
validation_x shape: (4134, 60, 8)
validation_y shape: (4134,)
Epoch 1/10


c:\Users\CRISTIAN CHAVEZ\Documents\GitHub\AI_Project_Deep-Learning-Tensorflow-Keras\env\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1288/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.5175 - loss: 0.7485
Epoch 1: val_accuracy improved from -inf to 0.54499, saving model to models/RNN_Final-01-0.545.keras
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 131s 99ms/step - accuracy: 0.5175 - loss: 0.7485 - val_accuracy: 0.5450 - val_loss: 0.6875
Epoch 2/10
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.5544 - loss: 0.6844
Epoch 2: val_accuracy did not improve from 0.54499
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 226s 176ms/step - accuracy: 0.5544 - loss: 0.6844 - val_accuracy: 0.5450 - val_loss: 0.6836
Epoch 3/10
1287/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.5765 - loss: 0.6764
Epoch 3: val_accuracy improved from 0.54499 to 0.55878, saving model to models/RNN_Final-03-0.559.keras
1288/1288 ━━━━━━━━━━━━━━━━━━━━ 252s 196ms/step - accuracy: 0.5765 - loss: 0.6764 - val_accuracy: 0.5588 - val_loss: 0.6844
Epoch 4/10
1287/1288 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - accuracy: 0.5887 - loss: 0.6695
Epoch 4: val_accuracy did not